In [1]:
from iminuit import Minuit
from iminuit.cost import LeastSquares
from scipy.stats import norm, chi2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [2]:
data = pd.read_csv('new-na-0cm.CSV', delimiter=';')
data

,Channel,BGO,CSI,LYSO
0,-90,25,310,6
1,-70,40,292,36
2,-50,88,335,122
3,-30,160,286,294
4,-10,272,315,717
...,...,...,...,...
2181,43530,0,0,0
2182,43550,0,0,0
2183,43570,0,0,0
2184,43590,0,0,0


In [3]:
channel = data['Channel']
counts_BGO = np.array(data['BGO'])
counts_LYSO = np.array(data['LYSO'])
counts_CSI = np.array(data['CSI'])

frequency_BGO = counts_BGO/577
frequency_LYSO = counts_LYSO/1207
frequency_CSI = counts_CSI/1213

In [4]:
fig = px.line(x=channel, y=frequency_BGO, title='BGO')
fig.show()

In [5]:
fig = px.line(x=channel, y=frequency_LYSO, title='LYSO')
fig.show()

In [6]:
fig = px.line(x=channel, y=frequency_CSI, title='CSI')
fig.show()

In [7]:
def repeat_fit(ls, left_bound, right_bound, mu, sigma, A, ampiezza_picco_limite, dis=False, limit_ADC=100):
    '''
    ampiezza picco limite = è la ampiezza dell'intorno di mu in cui è accettabile avere il picco
    '''
    peak_list = []
    error_peak_list = []
    sigma_list = []
    error_sigma_list = []
    area_list = []
    error_area_list = []
    for delta_ADC in np.arange(0, abs(right_bound-left_bound)/2+limit_ADC, 10):
        ls.mask = (channel > left_bound+delta_ADC) & (channel < right_bound-delta_ADC)
        m = Minuit(ls, mu=mu, sigma=sigma, A=A)
        m.migrad()
        m.hesse()
        p_value = (1. - chi2.cdf (m.fval, df = m.ndof))
        if p_value > 0.05 and m.valid == True:
            if m.values['mu'] < (mu+ampiezza_picco_limite/2) and m.values['mu'] > (mu-ampiezza_picco_limite/2):    
                peak_list.append(m.values['mu'])
                error_peak_list.append(m.errors['mu'])
                sigma_list.append(m.values['sigma'])
                error_sigma_list.append(m.errors['sigma'])
                area_list.append(m.values['A'])
                error_area_list.append(m.errors['A'])
                if dis==True: display(m)
        ls.mask = None
    for delta_ADC in np.arange(0, abs(right_bound-left_bound)/2+limit_ADC, 10):
        ls.mask = (channel > left_bound+delta_ADC) & (channel < right_bound-delta_ADC/2)
        m = Minuit(ls, mu=mu, sigma=sigma, A=A)
        m.migrad()
        m.hesse()
        p_value = (1. - chi2.cdf (m.fval, df = m.ndof))
        if p_value > 0.05 and m.valid == True:
            if m.values['mu'] < (mu+ampiezza_picco_limite/2) and m.values['mu'] > (mu-ampiezza_picco_limite/2):    
                peak_list.append(m.values['mu'])
                error_peak_list.append(m.errors['mu'])
                sigma_list.append(m.values['sigma'])
                error_sigma_list.append(m.errors['sigma'])
                area_list.append(m.values['A'])
                error_area_list.append(m.errors['A'])
                if dis==True: display(m)
        ls.mask = None
    for delta_ADC in np.arange(0, abs(right_bound-left_bound)/2+limit_ADC, 10):
        ls.mask = (channel > left_bound+delta_ADC/2) & (channel < right_bound-delta_ADC)
        m = Minuit(ls, mu=mu, sigma=sigma, A=A)
        m.migrad()
        m.hesse()
        p_value = (1. - chi2.cdf (m.fval, df = m.ndof))
        if p_value > 0.05 and m.valid == True:
            if m.values['mu'] < (mu+ampiezza_picco_limite/2) and m.values['mu'] > (mu-ampiezza_picco_limite/2):    
                peak_list.append(m.values['mu'])
                error_peak_list.append(m.errors['mu'])
                sigma_list.append(m.values['sigma'])
                error_sigma_list.append(m.errors['sigma'])
                area_list.append(m.values['A'])
                error_area_list.append(m.errors['A'])
                if dis==True: display(m)
        ls.mask = None
    return np.array(peak_list), np.array(error_peak_list), np.array(sigma_list), np.array(error_sigma_list), np.array(area_list), np.array(error_area_list)

In [8]:
def weighted_mean(values, errors):
    mean = np.sum(values/(errors**2))/np.sum(1/(errors**2))
    error_mean = abs(np.max(values) - np.min(values))/2 # errore sistematico dovuto alla scelta del range di interpolazione.
    # non è errore statistico perchè non è dato dalla ripetizione della misura
    return mean, error_mean

In [9]:
def weighted_mean_a(values, errors, central_value, margin):
    new_values = []
    new_errors = []
    for i in range(len(values)):
        if values[i] < central_value+margin and values[i] > central_value-margin:
            new_values.append(values[i])
            new_errors.append(errors[i])
    new_values = np.array(new_values)
    new_errors = np.array(new_errors)
    print(new_values)
    print(new_errors)
    mean = np.sum(new_values/(new_errors**2))/np.sum(1/(new_errors**2))
    error_mean = abs(np.max(new_values) - np.min(new_values))/2 # errore sistematico dovuto alla scelta del range di interpolazione.
    # non è errore statistico perchè non è dato dalla ripetizione della misura
    return mean, error_mean

In [10]:
def func(x, mu, sigma, A):
    return A * norm.pdf(x, mu, sigma)

In [11]:
error_BGO = np.sqrt(counts_BGO)/577
error_LYSO = np.sqrt(counts_LYSO)/1207
error_CSI = np.sqrt(counts_CSI)/1213

ls_BGO = LeastSquares(channel, frequency_BGO, error_BGO, func)
ls_LYSO = LeastSquares(channel, frequency_LYSO, error_LYSO, func)
ls_CSI = LeastSquares(channel, frequency_CSI, error_CSI, func)

In [12]:
peak_BGO, error_peak_BGO, sigma_BGO, err_sigma_BGO, a_BGO, err_a_BGO = repeat_fit(ls_BGO, 1900, 2800, 2400, 160, 50, 350)
mean_BGO, error_mean_BGO = weighted_mean(peak_BGO, error_peak_BGO)
area_BGO, error_area_BGO = weighted_mean_a(a_BGO, err_a_BGO, 20500, 1000)
print(mean_BGO, error_mean_BGO)
print(area_BGO, error_area_BGO)

[21396.18721609 21396.18721609 21117.93562073 21117.93562073
 21298.3664918  21298.3664918  20658.82590797 20658.82590797
 21179.05405396 21179.05405396 20265.47492389 20205.34561514
 20028.35285461 19999.66003926 19999.66003926 19698.91455731]
[ 150.5265025   150.5265025   224.98145084  224.98145084  397.35272215
  397.35272215  704.90798404  704.90798404 1904.32430394 1904.32430394
   70.76895728   76.79264597  137.48132596  182.81063911  182.81063911
  238.01396056]
2388.545057519307 114.10279915984506
20420.60662350078 848.6363293920331


In [42]:
peak_LYSO, error_peak_LYSO, sigma_LYSO, err_sigma_LYSO, a_LYSO, err_a_LYSO = repeat_fit(ls_LYSO, 5800, 8600, 7300, 600, 12, 500)
mean_LYSO, error_mean_LYSO = weighted_mean(peak_LYSO, error_peak_LYSO)
#area_LYSO, error_area_LYSO = weighted_mean_a(a_LYSO, err_a_LYSO, 13400, 1000)
print(mean_LYSO, error_mean_LYSO)
#print(area_LYSO, error_area_LYSO)

7367.28771050412 69.50975802434778


In [188]:
peak_CSI, error_peak_CSI, sigma_CSI, err_sigma_CSI, a_CSI, err_a_CSI = repeat_fit(ls_CSI, 12000, 15500, 14100, 1200, 5, 500)
mean_CSI, error_mean_CSI = weighted_mean(peak_CSI, error_peak_CSI)
area_CSI, error_area_CSI = weighted_mean_a(a_CSI, err_a_CSI, 5300, 1000)
print(mean_CSI, error_mean_CSI)
print(area_CSI, error_area_CSI)

[6001.05644059 6001.05644059 5773.26178108 5773.26178108 5798.62131978
 5837.98059087 5820.61317036 5820.61317036 5798.99177679]
[531.90647139 531.90647139 101.03368058 101.03368058 109.80485445
 119.04926843 126.45575167 126.45575167 142.82510425]
14095.274929008609 130.73684031866833
5802.406104748128 113.89732975806646
